In [269]:
%%time
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
CPU times: user 47.2 ms, sys: 5.5 ms, total: 52.7 ms
Wall time: 3.28 s


In [270]:
# machine specifications:
import platform
print(platform.machine())
print(platform.version())
print(platform.platform())
print(platform.uname())
print(platform.system())
print(platform.processor())

x86_64
#1 SMP PREEMPT_DYNAMIC Sat Nov 18 15:31:17 UTC 2023
Linux-6.1.58+-x86_64-with-glibc2.35
uname_result(system='Linux', node='45198cd6bcf3', release='6.1.58+', version='#1 SMP PREEMPT_DYNAMIC Sat Nov 18 15:31:17 UTC 2023', machine='x86_64')
Linux
x86_64


In [271]:
%%time
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install PyMuPDF
!pip install python-docx
!pip install beautifulsoup4 lxml
!pip install pytesseract
!sudo apt-get install tesseract-ocr
!pip install tensorflow
!pip install sentence_transformers
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 38.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
CPU times: user 772 ms, sys: 70.6 ms, total: 843 ms
Wall time: 1min 17s


In [272]:
%%time
import json
import re
import fitz
from docx import Document
from bs4 import BeautifulSoup
import pytesseract
from PIL import Image
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util

CPU times: user 1.4 ms, sys: 0 ns, total: 1.4 ms
Wall time: 12.2 ms


In [273]:
!pip install docx2pdf


In [274]:
%%time

from docx2pdf import convert

def doc2text(file_path) :

    if file_path.endswith('pdf'):
        with fitz.open(file_path) as doc:
            text = ""
            for page in doc:
                text += page.get_text()
        return text

    elif file_path.endswith('docx') or file_path.endswith('DOCX') or file_path.endswith('Docx'):
        document = Document(file_path)
        text = []
        for paragraph in document.paragraphs:
            text.append(paragraph.text)
        return '\n'.join(text)

    elif file_path.endswith('html') or file_path.endswith('HTML'):
        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()
        soup = BeautifulSoup(html_content, 'lxml')
        return soup.get_text(separator='\n', strip=True)

    elif file_path.endswith('jpg') or file_path.endswith('jpeg') or file_path.endswith('png'):
      image = Image.open(file_path)
      extracted_text = pytesseract.image_to_string(image)
      return extracted_text


CPU times: user 15 µs, sys: 0 ns, total: 15 µs
Wall time: 18.4 µs


# Resume Parsing

In [275]:
%%time
sec = [[#skills
        'areas of experience',
        'areas of expertise',
        'areas of knowledge',
        'skills',

        "other skills",
        "other abilities",
        'career related skills',
        'professional skills',
        'specialized skills',
        'technical skills',
        'soft skills',
        'computer skills',
        'personal skills',
        # 'computer knowledge',
        # 'technologies',

        'proficiencies',
        'languages',
        'language competencies and skills',
        'programming languages',
        'competencies'
],
      [#experience
        'employment history',
        'employment data',
        'career summary',
        'work history',
       'technical experience',
        'working history',
        'work experience',
        'experience',
        'professional experience',
        'professional background',
        'professional employment',
        'additional experience',
        'career related experience',
        "professional employment history",
        'related experience',
        'relevant experience',
        'programming experience',
        'freelance',
        'freelance experience',
        'internship experience',
        'internships',
        'apprenticeships',
        'army experience',
        'military experience',
        'military background',
               'special training',
       'training'

],
[#education
        'academic background',
        'academic experience',

        'courses',
        'qualification',
        'related courses',
        'education',
        'educational background',
        'educational qualifications',
        'educational training',
        'education and training',
        'academic training',
        'Academic Qualification',
        #'professional training',
        'course project experience',
        'related course projects',
        'college activities',
        'certifications',
    ],



[#miscellaneous
        'activities and honors',
        #'activities',
        'affiliations',
        'professional affiliations',
        'associations',
        'professional associations',
        'memberships',
        'professional memberships',
        'athletic involvement',
        'community involvement',

        'civic activities',
        'extra-Curricular activities',
        'professional activities',
        'volunteer work',
        'volunteer experience',
        'additional information',

],

[#achievements
        'achievement',
        'achievements',
        'awards and achievements',
        'licenses',
        'license',

        'conference presentations',
        'conventions',
        'dissertations',
        'exhibits',
        'papers',
        'publications',
        'professional publications',
        'research experience',
        'research grants',


        'research projects',
        'personal projects',
        'current research interests',

]
]

section_names=["skills","experience","education","miscellaneous","achievements"]
check_segmented = [
    False,
    False,
    False,
    False,
    False,

]

def classify_sections(text):
    # custom logic for classifying sections (similar to the previous example)
    sections = {}
    current_section = None
    temp=False
    for line in text.split('\n'):
     # print(line)
      temp=False
      for section_keywords in sec:
        if  (check_segmented[sec.index(section_keywords)]==False) and any((keyword.lower() in line.lower())  for keyword in section_keywords):
            for i in range(len(check_segmented)):
              check_segmented[i]=False
            check_segmented[sec.index(section_keywords)] = True

            temp=True
            current_section =section_names[sec.index(section_keywords)]
            if sections.get(current_section,0)!=0:
              continue
            sections[current_section] = []
            break
      if current_section !=None and temp==False:
        sections[current_section].append(line)


    return sections

def parse_resume(path):
    text = doc2text(path)
    sections = classify_sections(text)
    return text, sections


CPU times: user 10 µs, sys: 0 ns, total: 10 µs
Wall time: 13.4 µs


In [276]:
%%time
import pandas as pd
df=pd.DataFrame(columns=section_names)
df

CPU times: user 2.83 ms, sys: 0 ns, total: 2.83 ms
Wall time: 3.32 ms


,skills,experience,education,miscellaneous,achievements


In [277]:
%%time
import os
resume_content=''
for filename in os.listdir("/content/drive/MyDrive/hackathon/resume-data"):

    resume_content, sections = parse_resume('/content/drive/MyDrive/hackathon/Resume Validation Set/easy_resume_level1_a.pdf')
    df=df.append(sections,ignore_index=True)
    print("Resume Sections:")
    for section, content in sections.items():
      print(section.capitalize() + "   :")

      print('\n'.join(content))

      print("-----------------")
    break
    print("******************************************************************")

Resume Sections:
Education   :
Columbia University
New York, NY
Master of Science in Data Science, GPA: 4.08/4.00
Dec 2020
Coursework: Machine Learning, Applied Machine Learning, Applied Deep Learning, Statistical Inference & Modeling,
Personalization Theory, Natural Language Processing, Algorithms for Data Science, Computer Systems, Exploratory Data
Analysis and Visualization
Nirma University
Ahmedabad, India
Bachelor of Technology in Computer Engineering, GPA: 9.50/10, Rank: 2/900
May 2019
Coursework: Machine Learning, Deep Learning, Artificial Intelligence, Linear Algebra, Algorithms
-----------------
Experience   :
SLB Software Technology Innovation Center (STIC)
Menlo Park, CA
Data Scientist
Feb 2021 - Present
Physics-Informed Machine Learning and Time Series Analysis
● Developed Physics Informed Machine Learning based Hybrid Framework to create an advisory system that identifies the
regions with risky Stick/Slip conditions and outputs an optimal operating window for drilling the 

<timed exec>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


In [278]:
%%time
# Define a function to concatenate skill and experience for each row
def concatenate_skills_experience(row):
    # Convert the list of skills to a string
    skills_str = ', '.join(row['skills']) if isinstance(row['skills'], list) else str(row['skills'])
    # Convert the element in the 'experience' column to a string
    experience_str = ', '.join(row['experience']) if isinstance(row['experience'], list) else str(row['experience'])
    return skills_str + ', ' + experience_str

# Create a new DataFrame 'x' with the 'skills' and 'experience' columns
x = df[['skills', 'experience']].copy()

# Apply the concatenate_skills_experience function to each row and assign the result to column 'y'
x['y'] = x.apply(concatenate_skills_experience, axis=1)

# Drop the original 'skills' and 'experience' columns
x.drop(['skills', 'experience'], axis=1, inplace=True)

x


CPU times: user 5.28 ms, sys: 0 ns, total: 5.28 ms
Wall time: 5.25 ms


,y
0,"● Programming Languages: Python, SQL, R, Java,..."


# Extract Noun Phrases

In [279]:
%%time
import spacy
import pandas as pd

# Load the English language model in SpaCy
nlp = spacy.load("en_core_web_sm")

# Function to extract noun phrases from text using SpaCy
def extract_noun_phrases(text):
    doc = nlp(text)
    noun_phrases = [chunk.text for chunk in doc.noun_chunks]
    return noun_phrases

# Load the DataFrame 'x' created in the previous code
# Replace 'x.csv' with the actual file path if saving/loading from a file


# Extract noun phrases from the 'y' column of DataFrame 'x'
noun_phrases = []
for text in x['y']:
    noun_phrases.extend(extract_noun_phrases(text))

noun_phrases

CPU times: user 879 ms, sys: 14.9 ms, total: 894 ms
Wall time: 896 ms


['● Programming Languages',
 'Python',
 'SQL',
 'R',
 'Java',
 'C++',
 'C',
 'Tools',
 'Technologies',
 'Scikit-Learn',
 'NumPy',
 'Pandas',
 'Statsmodels',
 'PyTorch',
 'OpenCV',
 'Scipy',
 'Google BigQuery',
 'Oracle',
 'DS',
 'MongoDB',
 'Google Cloud Platform',
 'GitHub',
 'LaTeX',
 'SLB Software Technology Innovation Center',
 'STIC',
 'Menlo Park',
 'CA',
 'Data Scientist',
 'Feb 2021 - Present',
 'Physics-Informed Machine Learning',
 'Time Series Analysis',
 'Developed Physics Informed Machine Learning',
 'Hybrid Framework',
 'an advisory system',
 'that',
 'the, regions',
 'risky Stick/Slip conditions',
 'outputs',
 'an optimal operating window',
 'the future stands',
 'Devised Hybrid framework',
 'Physics Informed Machine Learning',
 'digitally generating LWD logs',
 'Gamma-Ray',
 'logs',
 'real-time',
 'efficiency',
 'robustness',
 'log collection process',
 'implementation',
 'Physics Inspired Machine Learning based toolbox',
 'Time Series data',
 'Computer Vision',
 'variou

In [280]:
%%time
import pandas as pd
df1 = pd.read_csv(r'/content/drive/MyDrive/hackathon/noun_phrases_1.csv')
df2 = pd.read_csv(r'/content/drive/MyDrive/hackathon/noun_phrases_2.csv')


# Append df2 to df1
merged_df = df1.append(df2)
merged_df

CPU times: user 7.2 ms, sys: 1.94 ms, total: 9.14 ms
Wall time: 16.1 ms


<timed exec>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


,noun_phrase,skills/non_skills
0,management decision-making,skills
1,Power bi,skills
2,Tableau,skills
3,visualization tools,skills
4,Snowflake,skills
...,...,...
1494,database,non_skills
1495,direct marketing,non_skills
1496,the firm’s top clients,non_skills
1497,nature,non_skills


In [281]:
%%time
merged_df = merged_df.sample(frac=1, random_state=42)
merged_df.reset_index(drop=True, inplace=True)

CPU times: user 2.72 ms, sys: 0 ns, total: 2.72 ms
Wall time: 2.94 ms


In [282]:
%%time
merged_df['noun_phrase']

CPU times: user 177 µs, sys: 8 µs, total: 185 µs
Wall time: 194 µs


0                           portfolio
1                                 Key
2                   various locations
3                                   ●
4                    Google Workspace
                    ...              
2493                              Apr
2494                          MongoDB
2495                      A/B Testing
2496    Hybrid Neural Network Library
2497        the subject matter expert
Name: noun_phrase, Length: 2498, dtype: object

# Skill Extraction

In [283]:
%%time
mapping = {'skills': 1, 'non_skills': 0}

merged_df[' skills/non_skills'] = merged_df[' skills/non_skills'].map(mapping)


CPU times: user 2.53 ms, sys: 0 ns, total: 2.53 ms
Wall time: 2.61 ms


In [284]:
%%time
merged_df

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 10.5 µs


,noun_phrase,skills/non_skills
0,portfolio,0
1,Key,0
2,various locations,0
3,●,0
4,Google Workspace,1
...,...,...
2493,Apr,0
2494,MongoDB,1
2495,A/B Testing,1
2496,Hybrid Neural Network Library,1


In [285]:
%%time
merged_df['noun_phrase'] = merged_df['noun_phrase'].astype(str)

CPU times: user 924 µs, sys: 0 ns, total: 924 µs
Wall time: 935 µs


# Model Training for skill extraction

In [286]:
%%time
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import SpatialDropout1D, LSTM, Conv1D, GlobalMaxPooling1D, Dense, Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
# Tokenize the noun phrases
tokenizer = Tokenizer()
tokenizer.fit_on_texts(merged_df['noun_phrase'])
X_sequences = tokenizer.texts_to_sequences(merged_df['noun_phrase'])
print(X_sequences)
# Pad sequences to ensure uniform length
max_length = 100
X_padded = pad_sequences(X_sequences, maxlen=max_length, padding='post')

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_padded, merged_df[' skills/non_skills'], test_size=0.2, random_state=42)

# Define the model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=max_length),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    # Flatten(),
    Dense(units=10, activation='relu'),
    Dense(units=1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
print(model.summary())

# Train the model
model.fit(X_train, y_train, epochs=30, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


In [287]:
%%time
model.save('/content/drive/MyDrive/hackathon/model.h5')
import tensorflow as tf

# model = tf.keras.models.load_model('/content/drive/MyDrive/hackathon/model.h5')


CPU times: user 151 ms, sys: 7.15 ms, total: 159 ms
Wall time: 990 ms


# Skill Extraction

In [288]:
%%time
text='\n'.join(df['skills'][0])
# text+=df['experience'][0]
text+='\n'.join(df['experience'][0])
text

CPU times: user 429 µs, sys: 0 ns, total: 429 µs
Wall time: 440 µs


'● Programming Languages: Python, SQL, R, Java, C++, C\n● Tools and Technologies: Scikit-Learn, NumPy, Pandas, Statsmodels, PyTorch, OpenCV, Scipy, Google BigQuery, Oracle\nDS, MongoDB, Google Cloud Platform, GitHub, LaTeX\n3\nSLB Software Technology Innovation Center (STIC)\nMenlo Park, CA\nData Scientist\nFeb 2021 - Present\nPhysics-Informed Machine Learning and Time Series Analysis\n● Developed Physics Informed Machine Learning based Hybrid Framework to create an advisory system that identifies the\nregions with risky Stick/Slip conditions and outputs an optimal operating window for drilling the future stands\n● Devised Hybrid framework using Physics Informed Machine Learning for digitally generating LWD logs (Gamma-Ray\nlogs) in real-time to increase efficiency and robustness of log collection process\n● Working towards implementation of Physics Inspired Machine Learning based toolbox for Time Series data\nComputer Vision\n● Researched and implemented various Computer Vision based 

In [289]:
%%time
def extract_skills(text):
    noun_phrases = extract_noun_phrases(text)
    noun_phrases = np.array(noun_phrases).astype(str)

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(noun_phrases)
    X_sequences = tokenizer.texts_to_sequences(noun_phrases)

    # Pad sequences to ensure uniform length
    max_length = 100
    X_padded = pad_sequences(X_sequences, maxlen=max_length, padding='post')

    # Reshape input data to add embedding dimension
    X_padded = np.expand_dims(X_padded, axis=-1)  # Add embedding dimension

    predictions = model.predict(X_padded)

    # Filter predicted skills based on the threshold
    threshold = 0.5  # Adjust as needed
    predicted_skills = [noun_phrases[i] for i, pred in enumerate(predictions) if pred > threshold]

    return predicted_skills


CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 14.5 µs


In [290]:
%%time
extracted_skills = extract_skills(text)

6/6 [==============================] - 0s 9ms/step
CPU times: user 394 ms, sys: 3.86 ms, total: 398 ms
Wall time: 377 ms


In [291]:
%%time
print(f'Extracted Skills: {extracted_skills}')

Extracted Skills: ['● Programming Languages', 'R', 'Scikit-Learn', 'Pandas', 'OpenCV', 'Scipy', 'MongoDB', 'Google Cloud Platform', 'GitHub', '3\nSLB Software Technology Innovation Center', 'STIC', 'Menlo Park', 'CA\nData Scientist\nFeb', 'Present\nPhysics-Informed Machine Learning and Time Series Analysis\n● Developed Physics Informed Machine Learning', 'Hybrid Framework', 'an advisory system', 'the\nregions', 'risky Stick/Slip conditions', '● Devised Hybrid framework', 'Physics Informed Machine Learning', 'Gamma-Ray\nlogs', 'log collection process', 'Physics Inspired Machine Learning based toolbox', 'Time Series data\nComputer Vision', 'various Computer Vision based use cases', 'state', 'Object Detection', 'Object Tracking', 'Pose Estimation', 'Fine-tuned different\nobject detection models', 'the pose estimation pipelines', 'increased performance', 'field datasets\nMedia Center', 'Graduate Research', 'automate process', 'slide analysis', 'collection', 'slides', 'image processing', 'M

In [292]:
%%time
final_skills = '\n'.join(extracted_skills)

CPU times: user 24 µs, sys: 1 µs, total: 25 µs
Wall time: 30 µs


# Abstractive Skill Extraction

In [293]:
%%time
nlp_model = spacy.load('/content/drive/MyDrive/hackathon/skills_model')
# text=df['skills']
doc = nlp_model(final_skills)
abstractive_skills=''
for ent in doc.ents:
  abstractive_skills+=ent.text
  print(f'{ent.label_.upper()}->\n {ent.text}')

CPU times: user 795 ms, sys: 27 ms, total: 822 ms
Wall time: 948 ms


In [294]:
doc

● Programming Languages
R
Scikit-Learn
Pandas
OpenCV
Scipy
MongoDB
Google Cloud Platform
GitHub
3
SLB Software Technology Innovation Center
STIC
Menlo Park
CA
Data Scientist
Feb
Present
Physics-Informed Machine Learning and Time Series Analysis
● Developed Physics Informed Machine Learning
Hybrid Framework
an advisory system
the
regions
risky Stick/Slip conditions
● Devised Hybrid framework
Physics Informed Machine Learning
Gamma-Ray
logs
log collection process
Physics Inspired Machine Learning based toolbox
Time Series data
Computer Vision
various Computer Vision based use cases
state
Object Detection
Object Tracking
Pose Estimation
Fine-tuned different
object detection models
the pose estimation pipelines
increased performance
field datasets
Media Center
Graduate Research
automate process
slide analysis
collection
slides
image processing
ML/DL
techniques
photographic images
35mm slides collection
● Formulated pipeline
entire manual process
citation
slides
(Precision
95%
NJ
Data Scien

In [295]:
%%time
final_designation = '\n'.join(df['experience'][0])
final_designation

CPU times: user 139 µs, sys: 6 µs, total: 145 µs
Wall time: 149 µs


'SLB Software Technology Innovation Center (STIC)\nMenlo Park, CA\nData Scientist\nFeb 2021 - Present\nPhysics-Informed Machine Learning and Time Series Analysis\n● Developed Physics Informed Machine Learning based Hybrid Framework to create an advisory system that identifies the\nregions with risky Stick/Slip conditions and outputs an optimal operating window for drilling the future stands\n● Devised Hybrid framework using Physics Informed Machine Learning for digitally generating LWD logs (Gamma-Ray\nlogs) in real-time to increase efficiency and robustness of log collection process\n● Working towards implementation of Physics Inspired Machine Learning based toolbox for Time Series data\nComputer Vision\n● Researched and implemented various Computer Vision based use cases to improve Health Safety and Environment\n● Evaluated 5+ state of the at solutions for Object Detection, Object Tracking, and Pose Estimation. Fine-tuned different\nobject detection models and integrated them into th

# Find Job titles in resume

In [296]:
%%time
store_designations = []

nlp_model_d = spacy.load('/content/drive/MyDrive/hackathon/designation_model')
doc = nlp_model_d(final_designation)
for ent in doc.ents:
    store_designations.append(ent.text)
    print(f'{ent.label_.upper():{30}}- {ent.text}')

DESIGNATION                   - Data Scientist
DESIGNATION                   - Graduate Research Assistant
DESIGNATION                   - Data Science Analyst
DESIGNATION                   - Data Scientist Intern
CPU times: user 1.03 s, sys: 37.1 ms, total: 1.07 s
Wall time: 1.19 s


In [297]:
%%time
store_designations

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.34 µs


['Data Scientist',
 'Graduate Research Assistant',
 'Data Science Analyst',
 'Data Scientist Intern']

# O-Net mapping for Standardised Job Title

In [298]:
%%time
# Get the file path
file_path = '/content/drive/MyDrive/hackathon/data.csv'

# Read the CSV file into a DataFrame
df_ = pd.read_csv(file_path)

# Get the desired column as a list
labels = df_['Title'].tolist()
codes = df_['O*NET Code'].tolist()
res = dict(zip(labels, codes))

CPU times: user 7.91 ms, sys: 0 ns, total: 7.91 ms
Wall time: 11.8 ms


In [299]:
%%time
model2 = SentenceTransformer('all-MiniLM-L6-v2')

CPU times: user 174 ms, sys: 13.9 ms, total: 188 ms
Wall time: 1.06 s


In [300]:
%%time
standard_job_titles = labels
standard_embeddings = model2.encode(standard_job_titles)

CPU times: user 6.86 s, sys: 45.3 ms, total: 6.9 s
Wall time: 8.66 s


In [301]:
%%time
extracted_job_titles = store_designations
extracted_embeddings = model2.encode(extracted_job_titles)

CPU times: user 43 ms, sys: 0 ns, total: 43 ms
Wall time: 43.3 ms


In [302]:
%%time
# Function to find the most similar standard job title for each extracted job title
def find_most_similar_jobs(extracted_embeddings, standard_embeddings, standard_job_titles):
    for extracted_embedding in extracted_embeddings:
        similarities = util.cos_sim(extracted_embedding, standard_embeddings)
        max_index = np.argmax(similarities)

        yield standard_job_titles[max_index]

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 11.7 µs


In [303]:
%%time
# Mapping each extracted job title to the most similar standard job title
mapped_job_titles = list(find_most_similar_jobs(extracted_embeddings, standard_embeddings, standard_job_titles))
for extracted, mapped in zip(store_designations, mapped_job_titles):
    print(f'Extracted: {extracted} -> Mapped: {mapped}')

Extracted: Data Scientist -> Mapped: Data Scientists
Extracted: Graduate Research Assistant -> Mapped: Social Science Research Assistants
Extracted: Data Science Analyst -> Mapped: Data Scientists
Extracted: Data Scientist Intern -> Mapped: Data Scientists
CPU times: user 11.2 ms, sys: 7 µs, total: 11.2 ms
Wall time: 16.1 ms


# Extracting Education

In [304]:
# %%time
# if not df['education'].empty:
#     # Drop NaN values and convert elements to strings
#     df['education'] = df['education'].dropna().astype(str)
#     final_education = '\n'.join(df['education'])
#     print(final_education)


# Extracting Timeline

# Mapping description and extracting skills for each job title

In [305]:
%%time
s=df['experience'][0]
s=' '.join(s)
l=[0]
for i in store_designations:
  l+=[s.index(i)]
l+=[len(s)]
l
se=[]
for i in range(1,len(l)-1):
  se+=[s[l[i]:l[i+1]]]
sk=[]
for d in se:
  sk+=[extract_skills(d)]
sk

3/3 [==============================] - 0s 19ms/step
CPU times: user 823 ms, sys: 14.6 ms, total: 838 ms
Wall time: 1.29 s


[['Hybrid Framework',
  'an advisory system',
  'that',
  'risky Stick/Slip conditions',
  'an optimal operating window',
  'Physics Informed Machine Learning',
  'digitally generating LWD logs',
  'Gamma-Ray logs',
  'real-time',
  'efficiency',
  'robustness',
  'log collection process',
  'Physics Inspired Machine Learning based toolbox',
  'Time Series data Computer Vision',
  'various Computer Vision based use cases',
  'Health Safety',
  '+ state',
  'Object Detection',
  'Fine-tuned different object detection models',
  'increased performance',
  'field datasets Media Center',
  'Art',
  'History'],
 ['Graduate Research Assistant Mar',
  '● Engineered way',
  'slide analysis',
  'slides',
  'ML/DL techniques',
  'photographic images',
  '35mm',
  'collection ● Formulated pipeline',
  'entire manual process',
  'slides',
  '95%',
  'Recall:82%',
  'Unilever Englewood Cliffs'],
 ['an application',
  'the feature extraction and data engineering process',
  'Process Analytics Engine

In [306]:
%%time
skill_preds=[]
for skill in sk:
  temp='\n'.join(skill)
  doc=nlp_model(temp)
  temp_skills=''
  for ent in doc.ents:
   temp_skills+=ent.text
  skill_preds.append(temp_skills)

CPU times: user 161 ms, sys: 1.85 ms, total: 162 ms
Wall time: 265 ms


# Resume Summary

In [307]:
df

,skills,experience,education,miscellaneous,achievements
0,"[● Programming Languages: Python, SQL, R, Java...",[SLB Software Technology Innovation Center (ST...,"[Columbia University, New York, NY, Master of ...",NaN,[document summarizing 89 research papers invol...


In [308]:
%%time
raw_exp='\n'.join(df.iloc[0]['experience'])
raw_exp

CPU times: user 528 µs, sys: 0 ns, total: 528 µs
Wall time: 10 ms


'SLB Software Technology Innovation Center (STIC)\nMenlo Park, CA\nData Scientist\nFeb 2021 - Present\nPhysics-Informed Machine Learning and Time Series Analysis\n● Developed Physics Informed Machine Learning based Hybrid Framework to create an advisory system that identifies the\nregions with risky Stick/Slip conditions and outputs an optimal operating window for drilling the future stands\n● Devised Hybrid framework using Physics Informed Machine Learning for digitally generating LWD logs (Gamma-Ray\nlogs) in real-time to increase efficiency and robustness of log collection process\n● Working towards implementation of Physics Inspired Machine Learning based toolbox for Time Series data\nComputer Vision\n● Researched and implemented various Computer Vision based use cases to improve Health Safety and Environment\n● Evaluated 5+ state of the at solutions for Object Detection, Object Tracking, and Pose Estimation. Fine-tuned different\nobject detection models and integrated them into th

# Replacing Job Titles with Standard O*Net Job Titles

In [309]:
%%time
def replace_strings(work_experience, original_strings, replacement_strings):
    """
    Replace occurrences of strings in 'original_strings' with corresponding strings in 'replacement_strings' in the 'work_experience' text.

    Parameters:
    - work_experience (str): The text containing the work experience.
    - original_strings (list of str): Strings to be replaced.
    - replacement_strings (list of str): Strings to replace with.

    Returns:
    - str: Updated work experience text.
    """
    for original, replacement in zip(original_strings, replacement_strings):
        work_experience = work_experience.replace(original, replacement)
    return work_experience

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 12.4 µs


In [310]:
%%time
final_exp = replace_strings(raw_exp, store_designations, mapped_job_titles)

CPU times: user 98 µs, sys: 0 ns, total: 98 µs
Wall time: 2 ms


In [311]:
%%time
if len(store_designations) > 0 and len(mapped_job_titles) > 0:
    for designation, standard_designation in zip(store_designations, mapped_job_titles):
        if designation in raw_exp:
            raw_exp = raw_exp.replace(designation, standard_designation)


CPU times: user 154 µs, sys: 0 ns, total: 154 µs
Wall time: 4 ms


In [312]:
%%time
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

CPU times: user 2.61 s, sys: 232 ms, total: 2.84 s
Wall time: 4.83 s


In [313]:
%%time
import re

def time_line():
  date_pattern = r'\b(?:\d{2}/\d{4}|\d{2}/\d{2}|\d{2}-\d{2}-\d{2}|\d{2}-\d{2}-\d{4})\b'


  #for designation
  date_with_designation = []
  for designations in store_designations:
    index = final_designation.index(designations)
    temp_str = final_designation[index : ]

    dates = re.findall(date_pattern, temp_str)
    if(len(dates) > 0):
      date_with_designation.append(dates[0])





  return date_with_designation

des_date = time_line()

from dateutil.parser import parse

# print(mapped_titles)
# print(des_date)


time = dict(zip(des_date, store_designations))
dates = [parse(date_str) for date_str in des_date]
sorted_dates_with_titles = sorted(zip(dates, store_designations))
sorted_dates = [date.strftime('%Y-%m-%d') for date, _ in sorted_dates_with_titles]
sorted_titles = [title for _, title in sorted_dates_with_titles]
sorted_time = dict(zip(sorted_dates, sorted_titles))

# print(sorted_time)

CPU times: user 1.59 ms, sys: 0 ns, total: 1.59 ms
Wall time: 1.6 ms


In [314]:
print('----------------------------')
print('RESUME DETAILS')
print('----------------------------')
print('----------------------------')
print('EDUCATION')
print('----------------------------')
education = df.iloc[0]['education']
if isinstance(education, float):
    # Handle the case where education is NaN or a float
    education_list = ['']
else:
    education_list = list(education)
print('\n'.join(education_list))
print('----------------------------')
print('----------------------------')
print('EXPERIENCE')
print('----------------------------')
print('DESIGNATIONS AS PER O*NET DATABASE:')
print('----------------------------')
# print('\n'.join(mapped_job_titles))
for jtitle, desc, skill_pred in zip(mapped_job_titles, se, skill_preds):
    print('***TITLE***')
    print(f'{jtitle} --> {res[jtitle]}')
    print('***DESCRIPTION***')
    print(desc)
    print('***PREDICTED SKILLS***')
    print(skill_pred)
    print('\n')

print('----------------------------')
# print('\n'.join(df.iloc[0]['experience']))
# print(final_exp.strip())
print('----------------------------')
print('ABSTRACTIVE SKILLS')
print('----------------------------')
print(abstractive_skills)
print('----------------------------')
print('----------------------------')
print('SKILLS')
print('----------------------------')
print('\n'.join(list(df.iloc[0]['skills'])))
print('----------------------------')
print('----------------------------')
print('ACHIEVEMENTS')
print('----------------------------')
achievements = df.iloc[0]['achievements']
if isinstance(achievements, float):
    # Handle the case where achievements is NaN or a float
    achievements_list = ['']
else:
    achievements_list = list(achievements)
print('\n'.join(achievements_list))
print('----------------------------')
print('----------------------------')
print('CAREER TRAJECTORY')
print('----------------------------')
for key in sorted_time:
    print(f'{key}--->{sorted_time[key]}')
print('----------------------------')
print('----------------------------')


----------------------------
RESUME DETAILS
----------------------------
----------------------------
EDUCATION
----------------------------
Columbia University
New York, NY
Master of Science in Data Science, GPA: 4.08/4.00
Dec 2020
Coursework: Machine Learning, Applied Machine Learning, Applied Deep Learning, Statistical Inference & Modeling,
Personalization Theory, Natural Language Processing, Algorithms for Data Science, Computer Systems, Exploratory Data
Analysis and Visualization
Nirma University
Ahmedabad, India
Bachelor of Technology in Computer Engineering, GPA: 9.50/10, Rank: 2/900
May 2019
Coursework: Machine Learning, Deep Learning, Artificial Intelligence, Linear Algebra, Algorithms
----------------------------
----------------------------
EXPERIENCE
----------------------------
DESIGNATIONS AS PER O*NET DATABASE:
----------------------------
***TITLE***
Data Scientists --> 15-2051.00
***DESCRIPTION***
Data Scientist Feb 2021 - Present Physics-Informed Machine Learning and 